In [1]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
url = "https://www.naukri.com/financial-analyst-jobs-in-mumbai?k=financial%20analyst&l=mumbai"

In [2]:
#creating a dictionary for storing the information after scraping
driver = webdriver.Chrome('C:\selenium\chromedriver.exe')


In [7]:
results = soup.find(class_='list')

In [24]:
job_elems = results.find_all('article', class_='jobTuple bgWhite br4 mb-8')

In [72]:
jobs={"url":[],
     "title":[],
     "company":[],
     "payment":[],
     "experience":[],
     "rating":[],
     "reviews":[]}

In [30]:
jobs={"url":[],
     "title":[],
     "company":[],
     "payment":[],
     "experience":[],
     "location":[],
     "skills":[],
     "rating":[],
     "reviews":[]}

driver = webdriver.Chrome('C:\selenium\chromedriver.exe')
for index in range(1, 20):
    
    driver.get("https://www.naukri.com/data-scientist-jobs-{}".format(index))
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    results = soup.find(class_='list')
    job_elems = results.find_all('article', class_='jobTuple bgWhite br4 mb-8')    
    for job_elem in job_elems:
        skills = []
        url = job_elem.find('a', class_='title fw500 ellipsis').get('href')
        jobs['url'].append(url)
        title = job_elem.find('a', class_='title fw500 ellipsis').get('title')
        jobs['title'].append(title)
        company = job_elem.find('a', class_='subTitle ellipsis fleft').get('title')
        jobs['company'].append(company)
        payment = job_elem.find('li', class_='fleft grey-text br2 placeHolderLi salary').text
        jobs['payment'].append(payment)
        experience = job_elem.find('li', class_='fleft grey-text br2 placeHolderLi experience')
        if experience is None:
            jobs['experience'].append('No required')
        else:
            jobs['experience'].append(experience.text)
        location = job_elem.find('li', class_='fleft grey-text br2 placeHolderLi location')
        if location is None:
            jobs['location'].append('No required')
        else:
            jobs['location'].append(location.text)
        skill_ = job_elem.find('ul', class_='tags has-description')
        skill_list = skill_.find_all('li', class_='fleft fs12 grey-text lh16 dot')
        for sk_l in skill_list:
            skills.append(sk_l.text)
        jobs['skills'].append(skills)
        rating = job_elem.find('span', class_='starRating fleft dot')
        if rating is None:
            rating = 'No tiene rating'
            jobs['rating'].append(rating)
        else:
            jobs['rating'].append(rating.text)
        reviews = job_elem.find('a', class_='reviewsCount ml-5 fleft blue-text')
        if reviews is None:
            reviews = 'No tiene reviews'
            jobs['reviews'].append(reviews)
        else:
            jobs['reviews'].append(reviews.text)        

df_jobs = pd.DataFrame(jobs)

In [29]:
df_jobs.iloc[1]['skills']

['Business Intelligence',
 'Artificial Intelligence',
 'Natural Language Processing',
 'Neural Networks',
 'Data Mining',
 'Machine Learning',
 'Deep Learning',
 'SQL']

In [35]:
df_jobs[df_jobs.duplicated(subset=["title", "company", "experience", "payment", "location"])].shape

(29, 9)

In [37]:
df_jobs.drop_duplicates(subset=["title", "company", "experience", "payment", "location"], keep="first", inplace=True)

In [41]:
df_jobs['location'] = df_jobs['location'].apply(lambda line: line.split(","))

In [43]:
df_jobs = df_jobs.apply(lambda x: x.astype(str).str.lower())